# Whole-genome analysis workflow

In [2]:
# ~2 minutes to install 
#%pip install -U --no-cache-dir scikit-learn scikit-optimize prefect prefect-ray ray plotly openpyxl shap lion_pytorch pytorch_tabnet xgboost neptune pyspark pyarrow dill fastnumbers

In [4]:
from prefect import task, flow
from prefect.task_runners import ConcurrentTaskRunner
from prefect_ray.task_runners import RayTaskRunner
import ray

import pandas as pd
import numpy as np

import logging

!export PREFECT_LOGGING_LEVEL="WARNING"
ray.shutdown()
parallelRunner = ray.init(
  configure_logging=True,
  logging_level=logging.ERROR,
)
parallelRunner

319.59s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Python version:,3.10.10
Ray version:,2.3.1


In [5]:
from sklearn.ensemble import (
    AdaBoostClassifier,
    RandomForestClassifier,
)
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from xgboost import XGBClassifier
from pytorch_tabnet.tab_model import TabNetClassifier
from lion_pytorch import Lion

from skopt.space import Categorical, Integer, Real

from env import neptune_api_token

RadialBasisSVC = SVC
RadialBasisSVC.__name__ = "RadialBasisSVC"

clearHistory = True
config = {
    "vcfLike": {
        "path": "../notebook/Variant_report_NUPs_fixed_2022-03-28.xlsx",  # variant call table with annotations
        "sheet": "all cases vs all controls",  # sheet name if Excel spreadsheet
        "indexColumn": [
            "chrom",
            "position",
            "Gene",
        ],  # header that indexes variants (set as list with multiple columns)
        "compoundSampleIdDelimiter": "__",  # delimiter for compound sample IDs in column names
        "compoundSampleIdStartIndex": 1,  # index of first sample ID in compound sample ID
        "binarize": True,  # binarize variants to 0/1, or sum to weigh allele frequency
        "minAlleleFrequency": 0.05,  # filter out variants with allele frequency less than this
        # 'alleleModel': ['dominant', 'recessive', 'overDominant'],  # biallelic allele models to test on gene sets
        "filters": {},
    },  # TODO handle genotypes from related individuals
    "geneSets": {},  # TODO gene sets
    "tracking": {
        "name": "Nucleoporin genes, cases <= 15% accuracy",  # name of the experiment
        "entity": "ejmockler",
        "project": "ALS-NUPS-60-discordantCases",
        "plotAllSampleImportances": True,  # if calculating Shapely explanations, plot each sample in Neptune
        "token": neptune_api_token,
        "remote": False,  # if True, log to Neptune
    },
    "clinicalTable": {
        "path": "../notebook/ACWM.xlsx",  # clinical data as Excel spreadsheet
        "idColumn": "ExternalSampleId",  # genotype ID header
        "uniqueIdColumn": "ExternalSubjectId",  # unique ID for each patient
        "labelColumn": "Subject Group",  # header that has case/control labels
        "controlLabels": [
            "Non-Neurological Control"
        ],  # these labels include external sample IDs (like 1000 Genomes)
        "caseLabels": [],  # "ALS Spectrum MND"
        "controlAlias": "control",
        "caseAlias": "case",
        "filters": "pct_european>=0.85",  # filter out nonhomogenous samples with less than 85% European ancestry
    },
    "externalTables": {
        "path": [
            "../notebook/igsr-1000 genomes phase 3 release.tsv",
            "../notebook/ALS-NUPS-2000__discordantSamples.tsv",
            "../notebook/ACWM_ethnicallyVariable.tsv",
            "../notebook/ACWM_ethnicallyVariable.tsv",
            "../notebook/igsr-1000 genomes phase 3 release.tsv",
        ],  # external sample table
        "label": [
            "control",
            "case",
            "case",
            "control",
            "control",
        ],  # case | control | TODO mixed
        "setType": ["crossval", "crossval", "holdout", "holdout", "holdout"],
        "idColumn": [
            "Sample name",
            "id",
            "ExternalSubjectId",
            "ExternalSubjectId",
            "Sample name",
        ],  # sample ID header
        "filters": [
            "`Superpopulation code`=='EUR' & `Population name`!='Finnish'",  # remove finnish samples due to unusual homogeneity (verify w/ PCA)
            "`testLabel`==1",
            "`Subject Group`=='ALS Spectrum MND'",
            "`Subject Group`=='Non-Neurological Control'",
            "`Superpopulation code`!='EUR' & `Population name`!='Finnish'",
        ],
    },
    "sampling": {
        "bootstrapIterations": 60,
        "crossValIterations": 10,  # number of validations per bootstrap iteration
        "holdoutSplit": 0.1,
        "lastIteration": 0,
    },
    "model": {
        "stack": {
            LinearSVC(): {
                "tol": Real(1e-6, 1e1, prior="log-uniform"),
                "C": Real(1e-4, 1e1, prior="log-uniform"),
            },
            RadialBasisSVC(probability=True, kernel="rbf"): {
                "tol": Real(1e-4, 1e1, prior="log-uniform"),
                "C": Real(1e-4, 1e1, prior="log-uniform"),
                "gamma": Categorical(["scale", "auto"]),
            },
            LogisticRegression(penalty="l2", solver="saga"): {
                "tol": Real(1e-6, 1e1, prior="log-uniform"),
                "C": Real(1e-4, 1e1, prior="log-uniform"),
            },
            MultinomialNB(): {"alpha": Real(1e-10, 1e1, prior="log-uniform")},
            AdaBoostClassifier(): {
                "n_estimators": Integer(25, 75),
                "learning_rate": Real(1e-6, 1e1, prior="log-uniform"),
            },
            XGBClassifier(): {
                "learning_rate": Real(1e-6, 1e1, prior="log-uniform"),
                "n_estimators": Integer(10, 100),
            },
            RandomForestClassifier(): {
                "n_estimators": Integer(75, 200),
            },
        },
        "hyperparameterOptimization": False,
        "calculateShapelyExplanations": True,
    },
}

 
async def remove_all_flows():
  from prefect.client import get_client
  orion_client = get_client()
  flows = await orion_client.read_flows()
  for flow in flows:
    flow_id = flow.id
    print(f"Deleting flow: {flow.name}, {flow_id}")
    await orion_client._client.delete(f"/flows/{flow_id}")
    print(f"Flow with UUID {flow_id} deleted")

if clearHistory: await remove_all_flows()

Deleting flow: processInputFiles, f3d3fb91-49eb-4ab4-9c00-cb4cb3cc8528
Flow with UUID f3d3fb91-49eb-4ab4-9c00-cb4cb3cc8528 deleted


In [2]:
from prefect import unmapped
from tqdm import tqdm

from input import processInputFiles

(caseGenotypes,
caseIDs,
holdoutCaseGenotypes,
holdoutCaseIDs,
controlGenotypes,
controlIDs,
holdoutControlGenotypes,
holdoutControlIDs,
clinicalData) = await processInputFiles(config)

print(f"\nclinical data:\n{clinicalData.head()}")

00:59:48.342 | INFO    | prefect.engine - Created flow run 'lurking-lion' for flow 'processInputFiles'

00:59:48.345 | INFO    | Flow run 'lurking-lion' - View at http://127.0.0.1:4200/flow-runs/flow-run/9b283d8a-1934-4759-996e-29a9b03eca7d

00:59:48.439 | INFO    | Flow run 'lurking-lion' - Created task run 'load-0' for task 'load'

00:59:48.441 | INFO    | Flow run 'lurking-lion' - Executing 'load-0' immediately...

/home/noot/.pyenv/versions/miniconda3-latest/lib/python3.10/site-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


...

01:04:33.506 | INFO    | Flow run 'lurking-lion' - 
684 controls:
 {'CGND-HDA-03798', 'NA12286', 'HG00099', 'NA12751', 'CGND-HDA-01426', 'HG01613', 'CGND-HDA-02728', 'HG01602', 'NA12775', 'NA07051', 'NA20762', 'NA20775', 'CGND-HDA-04090', 'NA10851', 'HG01630', 'HG01682', 'HG00159', 'HG00137', 'NA20511', 'CGND-HDA-03157', 'CGND-HDA-01641', 'NA11830', 'CGND-HDA-01427', 'CGND-HDA-00209', 'HG00143', 'CGND-HDA-03671', 'CGND-HDA-03167', 'NA12282', 'CGND-HDA-01852', 'NA12046', 'HG02224', 'CGND-HDA-03527', 'CGND-HDA-02774', 'HG00125', 'HG00150', 'HG00242', 'NA20587', 'NA11893', 'CGND-HDA-03739', 'NA07357', 'HG01765', 'HG01512', 'CGND-HDA-02518', 'NA20769', 'NA20795', 'HG01777', 'CGND-HDA-02772', 'HG01680', 'NA11932', 'HG00246', 'CGND-HDA-01593', 'CGND-HDA-03986', 'HG01625', 'HG01524', 'HG01773', 'HG01500', 'CGND-HDA-01454', 'NA20504', 'NA12761', 'CGND-HDA-03486', 'CGND-HDA-01515', 'HG00254', 'HG01775', 'CGND-HDA-02834', 'CGND-HDA-01414', 'HG01615', 'CGND-HDA-03668', 'CGND-HDA-03522', 'CGND-HDA-01487', 'HG01789', 'HG00148', 'CGND-HDA-03772', 'NA12348', 'CGND-HDA-03683', 'NA20815', 'NA07056', 'NA20522', 'NA20757', 'HG02220', 'HG01697', 'CGND-HDA-01608', 'CGND-HDA-01655', 'CGND-HDA-01640', 'NA12874', 'CGND-HDA-03147', 'CGND-HDA-03584', 'HG00157', 'CGND-HDA-03977', 'HG01536', 'CGND-HDA-01486', 'HG02231', 'CGND-HDA-00556', 'CGND-HDA-01522', 'HG00109', 'HG01528', 'CGND-HDA-01518', 'NA20521', 'NA11894', 'HG01605', 'HG00234', 'HG00243', 'CGND-HDA-03794', 'CGND-HDA-03476', 'HG00118', 'CGND-HDA-03579', 'NA10847', 'HG00258', 'CGND-HDA-03519', 'CGND-HDA-01692', 'CGND-HDA-01557', 'HG00238', 'NA20531', 'CGND-HDA-02762', 'CGND-HDA-03152', 'CGND-HDA-02838', 'NA12383', 'CGND-HDA-03479', 'NA12776', 'CGND-HDA-02327', 'HG01786', 'CGND-HDA-02835', 'CGND-HDA-01532', 'NA06984', 'NA20809', 'NA20756', 'HG01684', 'HG01768', 'NA20804', 'NA12287', 'CGND-HDA-01653', 'NA12815', 'CGND-HDA-03784', 'CGND-HDA-02833', 'NA20525', 'HG01632', 'HG00119', 'CGND-HDA-01123', 'CGND-HDA-01606', 'CGND-HDA-03777', 'CGND-HDA-03764', 'CGND-HDA-03792', 'CGND-HDA-01484', 'HG00233', 'HG00158', 'CGND-HDA-03470', 'CGND-HDA-00564', 'HG01530', 'HG01334', 'NA20766', 'NA12045', 'NA20828', 'NA20760', 'NA12044', 'NA20805', 'NA20773', 'HG01767', 'CGND-HDA-02075', 'HG01781', 'NA20763', 'HG01618', 'HG00112', 'HG00240', 'HG00106', 'CGND-HDA-02710', 'CGND-HDA-01464', 'HG00123', 'NA06985', 'NA20826', 'CGND-HDA-03759', 'HG01761', 'NA20832', 'HG00140', 'HG00113', 'CGND-HDA-03788', 'NA12273', 'HG00107', 'HG01679', 'HG00130', 'HG01757', 'NA12006', 'NA20800', 'HG00129', 'CGND-HDA-01651', 'CGND-HDA-03201', 'CGND-HDA-02183', 'HG01612', 'NA20767', 'NA12414', 'HG00101', 'CGND-HDA-03457', 'HG00139', 'CGND-HDA-03978', 'CGND-HDA-03144', 'HG01673', 'HG01519', 'NA11930', 'CGND-HDA-03079', 'NA12272', 'NA12003', 'HG00251', 'CGND-HDA-03548', 'CGND-HDA-03858', 'CGND-HDA-01953', 'CGND-HDA-03864', 'CGND-HDA-01605', 'HG00252', 'HG00174', 'NA20502', 'CGND-HDA-02088', 'NA20507', 'CGND-HDA-02706', 'HG01686', 'HG01510', 'CGND-HDA-03588', 'HG01700', 'NA12413', 'HG01503', 'HG01509', 'CGND-HDA-00434', 'NA11843', 'CGND-HDA-02836', 'CGND-HDA-02714', 'CGND-HDA-03783', 'NA12347', 'CGND-HDA-02742', 'HG01695', 'CGND-HDA-03487', 'CGND-HDA-02519', 'CGND-HDA-02516', 'CGND-HDA-03876', 'CGND-HDA-01901', 'CGND-HDA-01541', 'CGND-HDA-03813', 'NA20814', 'HG02215', 'CGND-HDA-03787', 'HG01747', 'NA20506', 'HG02232', 'NA11831', 'CGND-HDA-03520', 'HG01705', 'CGND-HDA-03197', 'HG01770', 'HG01672', 'NA20512', 'NA20532', 'NA20509', 'HG00121', 'CGND-HDA-02446', 'CGND-HDA-02520', 'CGND-HDA-01416', 'HG01628', 'NA20530', 'CGND-HDA-03689', 'NA20527', 'NA12763', 'HG01513', 'CGND-HDA-02448', 'NA12890', 'CGND-HDA-01604', 'CGND-HDA-03790', 'CGND-HDA-03785', 'NA12004', 'HG00142', 'HG00253', 'NA20754', 'HG00116', 'NA07000', 'NA20543', 'CGND-HDA-03951', 'CGND-HDA-03786', 'NA20818', 'CGND-HDA-03142', 'NA20827', 'CGND-HDA-03084', 'CGND-HDA-02189', 'CGND-HDA-01538', 'CGND-HDA-03524', 'NA20540', 'CGND-HDA-03103', 'CGND-HDA-03137', 'NA20799', 'HG01501', 'NA12760',

01:04:33.537 | INFO    | Flow run 'lurking-lion' - Filtered 1423 alleles with frequency below 5.000%

01:04:33.538 | INFO    | Flow run 'lurking-lion' - Kept 59 alleles

01:04:33.636 | INFO    | Flow run 'lurking-lion' - Finished in state Completed()


clinical data:
                       Quote    Data File ID ExternalSubjectId   
ExternalSampleId                                                 
CGND-HDA-05557    CGND_14852  CGND-HDA-05557       NEUUF013XXL  \
CGND-HDA-05556    CGND_14852  CGND-HDA-05556       NEUHM496PGR   
CGND-HDA-05555    CGND_14852  CGND-HDA-05555       NEUPK599KHH   
CGND-HDA-05554    CGND_14852  CGND-HDA-05554       NEUHD589CVP   
CGND-HDA-05553    CGND_14852  CGND-HDA-05553       NEUXX223WT8   

                              Project     Site Sample Collected   
ExternalSampleId                                                  
CGND-HDA-05557    ALS Natural History  Henry Ford Health System  \
CGND-HDA-05556    ALS Natural History  Henry Ford Health System   
CGND-HDA-05555    ALS Natural History  Henry Ford Health System   
CGND-HDA-05554    ALS Natural History  Henry Ford Health System   
CGND-HDA-05553    ALS Natural History  Henry Ford Health System   

                   Site Specimen Collected     Sex 

## Evaluate model stack

In [3]:
import neptune
from sklearn.model_selection import StratifiedKFold
from entrypoint import classify

outerCvIterator = StratifiedKFold(
    n_splits=config["sampling"]["crossValIterations"], shuffle=False
)
innerCvIterator = outerCvIterator
if config["tracking"]["remote"]:
    projectTracker = neptune.init_project(
        project=f'{config["tracking"]["entity"]}/{config["tracking"]["project"]}',
        api_token=config["tracking"]["token"],
    )

results = []
for model, hyperParameterSpace in list(config["model"]["stack"].items()):
    results.append(
        await classify(
            caseGenotypes,
            controlGenotypes,
            holdoutCaseGenotypes,
            holdoutControlGenotypes,
            clinicalData,
            model,
            hyperParameterSpace,
            innerCvIterator,
            outerCvIterator,
        ),
    )

NameError: name 'config' is not defined